In [5]:
from Bio import SeqIO

In [5]:
!pip install memory_profiler

  Obtaining dependency information for memory_profiler from https://files.pythonhosted.org/packages/49/26/aaca612a0634ceede20682e692a6c55e35a94c21ba36b807cc40fe910ae1/memory_profiler-0.61.0-py3-none-any.whl.metadata
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [37]:
from transformers import EsmForSequenceClassification, EsmModel, AutoConfig, EsmConfig, EsmForTokenClassification
from peft import PeftModel
import torch
import esm
from Bio import SeqIO
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from Bio.Seq import Seq
import logging
from tqdm import tqdm
import gc
from memory_profiler import profile
import statistics
import psutil
import time
from scipy.stats import gaussian_kde
import torch
from collections import Counter
gc.collect()


1151

In [2]:
from src.unique_virus_fasta import proteins_fasta, virus_fasta
from src.unique_virus_interactions import full_interactions
from src.get_attentions_viruses import parse_fasta, process_sequences
from src.distance_scores import *
from src.virus_contacts_numbers import count_contacts
from src.analyse_contacts_virus import all_metrics, read_full_interaction_file, read_interaction_file


In [6]:
def apc(x):
    a1 = x.sum(-1, keepdims=True)
    a2 = x.sum(-2, keepdims=True)
    a12 = x.sum((-1, -2), keepdims=True)
    avg = a1 * a2
    avg.div_(a12)  # in-place to reduce memory
    x.sub_(avg) # in-place to reduce memory
    del avg
    return x

In [28]:
def get_attention_block(self_attention_t_weighted, p1_boundaries, p2_boundaries, apc_norm=True):
    """
        self_attention: square matrix representing the self attentions
        returns: mutal attention between p1 and p2        
    """
    #self_attention_t_weighted_reduced = self_attention_t_weighted[:, 1:-1, 1:-1]


    block1 = self_attention_t_weighted[0][p1_boundaries[0]: p1_boundaries[1], p2_boundaries[0]: p2_boundaries[1]]
    block2 = self_attention_t_weighted[0][p2_boundaries[0]: p2_boundaries[1], p1_boundaries[0]: p1_boundaries[1]]

    block2_transposed = block2.t()
    mutual_information_tensor = block1 + block2_transposed
    mutual_apc = apc((mutual_information_tensor))    
    return mutual_apc



In [230]:
attention_block.nanmean()

tensor(-0.0734, grad_fn=<DivBackward0>)

In [276]:
attention_block = torch.load("./sequence_0_mutual_information/4_5_attention.pt") # 295 -> 54
attention_block = torch.load("./sequence_0_mutual_information/10_11_attention.pt") # 926 ->  574
attention_block = torch.load("./sequence_0_mutual_information/5_9_attention.pt") #  255 -> 38
attention_block = torch.load("./sequence_0_mutual_information/1_9_attention.pt") #  177 -> 21
attention_block = torch.load("./sequence_0_mutual_information/4_10_attention.pt") # 794 -> 94
attention_block = torch.load("./sequence_0_mutual_information/0_3_attention.pt") # 808 -> 285
attention_block = torch.load("./sequence_0_mutual_information/1_5_attention.pt") #  741 -> 114
attention_block = torch.load("./sequence_0_mutual_information/2_9_attention.pt") #  217 -> 8
attention_block = torch.load("./sequence_0_mutual_information/1_2_attention.pt") # 736  -> 24

attentions = np.array(attention_block.reshape(1, -1).tolist()[0])
threshold = np.mean(attentions) + (4.5 * np.std(attentions) )
len(attentions[attentions >= threshold]) / len(attentions)

0.009000163639338896

In [256]:
np.mean(attentions) + (3 * np.std(attentions) )
len(attentions[attentions >= 0.086])

866

In [250]:
attentions = np.array(attention_block.reshape(1, -1).tolist()[0])
interval_len = 0.001
x = np.arange(min(attentions),max(attentions), interval_len)
g_kde = gaussian_kde(attentions)
y = g_kde.evaluate(x)


prob_masses = y * interval_len
sorted(prob_masses)

[9.258701001459331e-56,
 1.1773857154429656e-54,
 1.7358812781035116e-54,
 3.516892400222404e-53,
 5.157845599134688e-53,
 9.974480186882855e-52,
 1.4540222799248563e-51,
 2.68223415724117e-50,
 3.88641080158364e-50,
 6.838765957487966e-49,
 9.84920105863696e-49,
 1.653230574961449e-47,
 2.366616901047598e-47,
 3.789341541760756e-46,
 5.391748005691111e-46,
 8.235096798264633e-45,
 1.1646780342253094e-44,
 1.6968689421259597e-43,
 2.3853761154137847e-43,
 3.315144997457057e-42,
 4.6321492380103406e-42,
 6.140891027866664e-41,
 8.528701788429601e-41,
 1.0785366020413418e-39,
 1.488874334614111e-39,
 1.7960276597407236e-38,
 2.464381322267814e-38,
 2.8357359451764024e-37,
 3.867519861587715e-37,
 4.245151469981479e-36,
 5.754823536788225e-36,
 6.025529656906466e-35,
 8.119070788458156e-35,
 8.109085899082742e-34,
 1.0860643486635268e-33,
 1.034720869326818e-32,
 1.3774624494579872e-32,
 1.2518409415217607e-31,
 1.6564527231126188e-31,
 1.4359844223664335e-30,
 1.8886587418644995e-30,
 1.

In [216]:
def get_highest_attn(attention_block, threshold=0.99, plot=False):
    """        
    _tensor: the symmetrized APC normalized attention between two proteins
    threshold: the p-value to consider
    """
    attentions = np.array(attention_block.reshape(1, -1).tolist()[0])
    interval_len = 0.001
    x = np.arange(min(attentions),max(attentions), interval_len)
    g_kde = gaussian_kde(attentions)
    y = g_kde.evaluate(x)


    prob_masses = y * interval_len
    assert np.isclose(sum(prob_masses), 1, 1e-3)

    cum_prob_masses = np.cumsum(prob_masses)
    index_first_outlier = np.where(cum_prob_masses >=threshold)[0][0]
    attention_threshold = x[index_first_outlier]
    all_outliers =  attentions[attentions > attention_threshold]
    a, b = np.where(attention_block > attention_threshold)
    return list(zip(a,b))    
    
#     if plot : 
#         plt.plot(x, eval)
#         plt.axvline(x=x[last_index_limit], color='black', linestyle='--', label='0.95')
#         plt.tight_layout()
#         plt.show()

        

    
    

In [300]:
all_seqs = SeqIO.to_dict(SeqIO.parse("/Users/mahdi/Documents/GitHub/phage_proj_2/ENA_clusters/ENA_votu_embedding_formatted.pep", 'fasta'))
seqs = SeqIO.to_dict(SeqIO.parse("./test_files/ENA_pol_rnr_assembled.fasta", 'fasta'))


current_seq_id = 0
seq_id2name = dict(enumerate(seqs.keys()))
seq_id = seq_id2name[current_seq_id]
lengths = list(map(int, seq_id.split("prots")[1][1:-1].split("_")))
print(len(lengths))
print(sum(lengths))

assert sum(lengths) == len(seqs[seq_id])


9
2346


In [301]:
seq_id

'ENA_AP011616_AP011616_1_30378_32498_33_to_ENA_AP011616_AP011616_1_35201_37594_41_prots_706_96_88_92_109_142_142_174_797_'

In [304]:
ls test_files/

ENA_pol_rnr_assembled.fasta             pol_rnr_3_full_attentions_weighted.pt
ena_id_to_protein.tsv                   pol_rnr_42_full_attentions_weighted.pt
pol_rnr_0_embeddings.pt                 pol_rnr_44_full_attentions_weighted.pt
pol_rnr_0_full_attentions_weighted.pt   pol_rnr_46_full_attentions_weighted.pt
pol_rnr_14_full_attentions_weighted.pt  pol_rnr_48_full_attentions_weighted.pt
pol_rnr_15_full_attentions_weighted.pt  pol_rnr_49_full_attentions_weighted.pt
pol_rnr_16_full_attentions_weighted.pt  pol_rnr_52_full_attentions_weighted.pt
pol_rnr_17_full_attentions_weighted.pt  pol_rnr_5_full_attentions_weighted.pt
pol_rnr_23_full_attentions_weighted.pt  pol_rnr_7_full_attentions_weighted.pt
pol_rnr_26_full_attentions_weighted.pt  pol_rnr_8_full_attentions_weighted.pt
pol_rnr_27_full_attentions_weighted.pt  pol_rnr_9_full_attentions_weighted.pt
pol_rnr_2_full_attentions_weighted.pt   pola_main_ref.pt
pol_rnr_34_full_attentions_weighted.pt  pola_seq.pt
pol_rnr_37_full_attentions_

In [307]:
!scp /Users/mahdi/Documents/GitHub/ViralEmbed/test_files/pol_rnr_{current_seq_id}_full_attentions_weighted.pt banane:/tmp/

pol_rnr_0_full_attentions_weighted.pt         100%   21MB  27.8MB/s   00:00    


In [305]:
p1_start, p1_end, p2_start, p2_end = 0, lengths[0],  sum(lengths[:-4]), sum(lengths[:-2])
# p1_start, p1_end, p2_start, p2_end = sum(lengths[:1]), sum(lengths[:2]),  sum(lengths[:-1]), sum(lengths)

segment_attentions = torch.load(f"./test_files/pol_rnr_{current_seq_id}_full_attentions_weighted.pt")

attention_block = get_attention_block(segment_attentions, (p1_start, p1_end), (p2_start, p2_end), apc_norm=True)
print(attention_block.shape, attention_block.shape[0] * attention_block.shape[1])
assert sorted(attention_block.shape) == sorted([lengths[0], lengths[-3]])

torch.Size([706, 797]) 562682


In [292]:
x_attention_sites = get_highest_attn(attention_block, threshold=0.999, plot=False)
x_attention_sites

[(0, 174),
 (0, 550),
 (0, 584),
 (1, 174),
 (5, 559),
 (6, 281),
 (6, 284),
 (7, 77),
 (7, 78),
 (7, 79),
 (7, 80),
 (7, 82),
 (7, 83),
 (7, 84),
 (7, 85),
 (7, 86),
 (7, 87),
 (7, 90),
 (7, 91),
 (7, 92),
 (7, 93),
 (7, 94),
 (7, 95),
 (7, 96),
 (7, 97),
 (7, 98),
 (7, 99),
 (7, 100),
 (7, 101),
 (7, 102),
 (7, 103),
 (7, 104),
 (7, 105),
 (7, 106),
 (7, 107),
 (7, 108),
 (7, 109),
 (7, 110),
 (7, 111),
 (7, 112),
 (7, 113),
 (7, 114),
 (7, 115),
 (7, 116),
 (7, 117),
 (7, 119),
 (7, 120),
 (7, 121),
 (7, 122),
 (7, 123),
 (7, 124),
 (7, 126),
 (7, 127),
 (7, 128),
 (7, 129),
 (7, 130),
 (7, 131),
 (7, 132),
 (7, 133),
 (7, 134),
 (7, 135),
 (7, 136),
 (7, 137),
 (7, 138),
 (7, 139),
 (7, 140),
 (7, 141),
 (7, 142),
 (7, 143),
 (7, 144),
 (7, 145),
 (7, 146),
 (7, 147),
 (7, 149),
 (7, 150),
 (7, 153),
 (7, 155),
 (7, 156),
 (7, 160),
 (7, 161),
 (7, 162),
 (7, 164),
 (7, 165),
 (7, 166),
 (7, 172),
 (7, 173),
 (7, 174),
 (7, 175),
 (7, 176),
 (7, 177),
 (7, 180),
 (9, 663),
 (9, 664

In [299]:
np.mean(sorted([segment_attentions[0][x] for x in x_attention_sites],  reverse=True))

0.060989518

In [279]:
x_attention_sites

[(239, 0),
 (243, 0),
 (243, 2),
 (245, 2),
 (247, 0),
 (247, 1),
 (247, 2),
 (247, 3),
 (247, 4),
 (247, 5),
 (247, 6),
 (247, 7),
 (248, 0),
 (250, 2),
 (250, 3),
 (250, 4),
 (250, 5),
 (250, 6),
 (250, 7),
 (250, 8),
 (250, 9),
 (250, 10),
 (251, 3),
 (251, 5)]